In [39]:
print("OK")

OK


In [40]:
%pwd

'd:\\Generative AI\\Langchain-Project\\Law-Assistant'

In [41]:
import os
os.chdir("../")

In [42]:
%pwd

'd:\\Generative AI\\Langchain-Project'

In [43]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [44]:
from langchain_community.document_loaders import CSVLoader, DirectoryLoader, PyPDFLoader
from langchain.docstore.document import Document
import os

# Load all CSV files first
def load_csv_documents(folder_path):
    csv_docs = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            loader = CSVLoader(file_path=os.path.join(folder_path, file))
            csv_docs.extend(loader.load())
    return csv_docs

# Load all PDFs after
def load_pdf_documents(folder_path):
    loader = DirectoryLoader(
        folder_path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    return loader.load()

# Combine all documents: CSV first, then PDFs
def load_all_documents(folder_path):
    csv_docs = load_csv_documents(folder_path)
    pdf_docs = load_pdf_documents(folder_path)
    combined = csv_docs + pdf_docs  # CSV comes first
    print(f"Loaded {len(csv_docs)} CSV chunks and {len(pdf_docs)} PDF chunks")
    return combined


In [45]:
docs = load_all_documents("./")  # Root directory

Loaded 0 CSV chunks and 0 PDF chunks


In [46]:
print("Loaded document count:", len(docs))
if not docs:
    print("No documents were loaded.")


Loaded document count: 0
No documents were loaded.


In [47]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embedding_model = download_hugging_face_embeddings()


In [48]:
query_result = embedding_model.embed_query("Hello world")
print("Length:", len(query_result))

Length: 384


In [49]:
# query_result

In [50]:
from dotenv import load_dotenv
load_dotenv()

False

In [51]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [53]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name and parameters
index_name = "lawbot"

# Create index
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '3a4f74a17923a7f025f1285bd38ddacf', 'date': 'Mon, 28 Jul 2025 07:21:09 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [54]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Set existing index name
index_name = "lawbot"

# No need to create it again — it already exists
# Just connect to it when needed


In [55]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [4]:
import os
from langchain_community.document_loaders import PyPDFLoader, CSVLoader, DirectoryLoader

def load_all_documents(folder_path):
    csv_docs = []
    pdf_docs = []

    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file.endswith(".csv"):
            loader = CSVLoader(file_path=file_path)
            csv_docs.extend(loader.load())
        elif file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
            pdf_docs.extend(loader.load())

    print(f"Loaded {len(csv_docs)} CSV chunks and {len(pdf_docs)} PDF chunks")
    return csv_docs + pdf_docs


In [5]:
docs = load_all_documents("../Data/")


Loaded 77104 CSV chunks and 109 PDF chunks


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

print(f"Total final chunks: {len(chunks)}")


Total final chunks: 115242


In [ ]:
from langchain_pinecone import PineconeVectorStore

batch_size = 200
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i+batch_size]
    PineconeVectorStore.from_documents(
        documents=batch,
        index_name=index_name,
        embedding=embedding_model
    )


In [10]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define index name and embedding model
index_name = "lawbot"  # replace with your actual Pinecone index name
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load the existing Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model
)



C:\Users\Rafiqul Islam\AppData\Local\Temp\ipykernel_28600\863623149.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [11]:
docsearch

In [14]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


In [15]:
retrieved_docs = retriever.invoke("What is penal code OF OFFENCES RELATING TO THE ARMY, NAVY AND AIR FORCE?")


In [16]:
retrieved_docs

[Document(id='e28ae4f3-10db-406b-8145-cfa0eabbc7ab', metadata={'row': 417.0, 'source': 'Data/bdlaws_ds_formatted.csv'}, page_content=': 417\nlaw_descripton: \nlaw_pass_date: 6th October, 1860\nlaw_subtitle: \nlaw_title: The Penal Code, 1860\nsection_chapter_id: 3.0\nsection_chapter_name: OF OFFENCES RELATING TO THE ARMY, NAVY AND AIR FORCE\nsection_chapter_no: Chapter VII\nsection_description: 139. No person subject to the\nsection_id: 2862\nsection_name: Persons subject to certain acts\nurl_id: 11\nvol_ordinance: ACT NO. XLV OF 1860'),
 Document(id='55fb4930-85a8-4ecd-9f8c-9e6653f1b867', metadata={'row': 449.0, 'source': 'Data/bd_laws_translated_05022022.csv'}, page_content=': 449\nUnnamed: 0: 449.0\nUnnamed: 0.1: 449.0\nlaw_descripton: \nlaw_pass_date: 6th October, 1860\nlaw_subtitle: -\nlaw_title: The Penal Code, 1860\nsection_chapter_id: 3.0\nsection_chapter_name: OF OFFENCES RELATING TO THE ARMY, NAVY AND AIR FORCE\nsection_chapter_no: Chapter VII\nsection_description: 135. Whoeve

In [18]:
from langchain_openai import OpenAI

# Initialize OpenAI LLM
llm = OpenAI(
    temperature=0.4,        # Controls creativity (lower = more focused)
    max_tokens=500          # Maximum number of tokens to generate
)

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_core.documents import Document

# 1. Define system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)

# 2. Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


In [22]:
# Create the question-answering chain using the prompt and LLM
question_answer_chain = create_stuff_documents_chain(
    llm,
    prompt
)

# Create the RAG (Retrieval-Augmented Generation) chain using retriever
rag_chain = create_retrieval_chain(
    retriever,
    question_answer_chain
)


In [27]:
try:
    response = rag_chain.invoke({"input": "What is the penal code OF OFFENCES RELATING TO THE ARMY, NAVY AND AIR FORCE?"})
    print("Answer:", response["answer"])
except Exception as e:
    print("⚠️ OpenAI Error:", e)


⚠️ OpenAI Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
